Análise da relação entre índices de PIB per capita e incidência do Covid-19 no estado do Rio de Janeiro

In [ ]:
#Parte comum:

import pandas as pd
import geopandas as gpd
# from jupyterthemes import jtplot
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as ps
import basedosdados as bd


states = bd.read_sql(
    '''
    SELECT DISTINCT id_municipio AS id, sigla_uf
    FROM  basedosdados.br_bd_diretorios_brasil.setor_censitario AS censo
    GROUP BY id_municipio, sigla_uf
    ''',
    billing_project_id='geobr-analysis')

cities = gpd.read_file("geojson\\brasil.json", encoding='utf-8')

brasil_data = pd.merge(states, cities, on='id')
brasil_data = gpd.GeoDataFrame(brasil_data, geometry='geometry')

RJ_data = brasil_data[brasil_data["sigla_uf"] == "RJ"].reset_index(drop=True)
RJ_data = RJ_data.astype({"id": int})

covid = pd.read_csv('caso_full.csv.gz', compression = 'gzip')
covid.rename(columns={"city_ibge_code": "id", "last_available_confirmed_per_100k_inhabitants": "cases_per_100k", "last_available_death_rate": "death_rate"}, inplace=True)
covid = covid[covid["id"] > 1000].astype({"id": int})
covid_rj = covid[(covid["state"] == "RJ") & (covid["date"] == "2021-11-17")].reset_index(drop=True)

covid_rj = covid_rj[["id", "estimated_population", "cases_per_100k", "death_rate"]]

RJ_fulldata = pd.merge(RJ_data, covid_rj, on='id')


In [ ]:
#Base de dados IBGE - PIB

pib = bd.read_sql(
    '''
    SELECT id_municipio as id, pib, ano
    FROM `basedosdados.br_ibge_pib.municipio`
    WHERE ano = 2018
    ''',
    billing_project_id='geobr-analysis')

pop = bd.read_sql(
    '''
    SELECT id_municipio as id, populacao as pop
    FROM `basedosdados.br_ibge_populacao.municipio`
    WHERE ano = 2018 AND sigla_uf = 'RJ'
    ''',
    billing_project_id='geobr-analysis')

In [ ]:
pib_data =  pib.astype({"id": int})
pop_data =  pop.astype({"id": int})

pib_data = pd.merge(pib_data, pop_data, on='id')

RJ_pib = pd.merge(RJ_fulldata, pib_data, on='id')

RJ_pib["per_capita"] = RJ_pib["pib"]/RJ_pib["pop"]

Agora, vamos plotar as informações. Na escala de cores, usaremos apenas até o valor de 100k, pois caso usássemos o maior valor outlier (200k), o mapa praticamente inteiro ficaria com cores claras demais.

In [ ]:
fig = px.choropleth(RJ_pib,
                   geojson = RJ_pib["geometry"],
                   locations = RJ_pib.index,
                   color = "per_capita",
                   hover_name = "name",
                   color_continuous_scale = "Greens",
                   range_color=(10000, 100000)
)

fig.update_geos(
                projection=dict(
                    scale=55
                    ),
                center = dict(
                    lat=-22.208333,
                    lon=-42.896388
                    )
)

fig.update_layout(
    title_text = "PIB per capita estimado de 2018, por município",
    margin={"r":0,"t":50,"l":0,"b":50}
)

fig.show()


In [ ]:
#checar regressao linear, etc.

fig = px.scatter(RJ_pib, x="death_rate", y="per_capita", hover_data=['name'], trendline="ols", color_discrete_sequence=['green'],trendline_color_override="orange")
fig.show()